In [4]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd 
import numpy as np 
from sklearn.dummy import  DummyRegressor
from sklearn.tree import  DecisionTreeRegressor
from sklearn.metrics import  mean_absolute_error as mae
from sklearn.model_selection import  cross_val_score
import eli5
from eli5.sklearn import PermutationImportance

In [8]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/part2/dw_part2/data'

/content/drive/My Drive/Colab Notebooks/dw_matrix/part2/dw_part2/data


In [0]:
df = pd.read_hdf('car.h5')

Dummy model test


In [12]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [0]:
feats=['car_id']
X=df[feats].values
y=df['price_value'].values

In [16]:
model=DummyRegressor()
model.fit(X,y)
y_pred=model.predict(X)
mae(y,y_pred)

39465.934630440985

In [18]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [20]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [21]:
df=df[df['price_currency'] != 'EUR']
df.shape

(106290, 155)

**Features**
 

In [22]:
df['param_color'].factorize()[0]

array([-1, -1, -1, ..., -1, -1, -1])

In [32]:
SUFFIX='__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list):
    continue
  if SUFFIX in feat: continue
  else: df[feat+SUFFIX]=df[feat].factorize()[0]
  print(df[feat+SUFFIX].head(1))

0    0
Name: created_at__cat, dtype: int64
0    0
Name: price_currency__cat, dtype: int64
0    0
Name: price_details__cat, dtype: int64
0    0
Name: price_value__cat, dtype: int64
0    0
Name: seller_address__cat, dtype: int64
0    0
Name: seller_name__cat, dtype: int64
0    0
Name: seller_type__cat, dtype: int64
0    0
Name: feature_czujniki-parkowania-przednie__cat, dtype: int64
0    0
Name: feature_poduszka-powietrzna-chroniąca-kolana__cat, dtype: int64
0    0
Name: feature_kurtyny-powietrzne__cat, dtype: int64
0    0
Name: feature_klimatyzacja-dwustrefowa__cat, dtype: int64
0    0
Name: feature_światła-led__cat, dtype: int64
0    0
Name: feature_czujnik-zmierzchu__cat, dtype: int64
0    0
Name: feature_elektrycznie-ustawiane-lusterka__cat, dtype: int64
0    0
Name: feature_asr-(kontrola-trakcji)__cat, dtype: int64
0    0
Name: feature_poduszka-powietrzna-kierowcy__cat, dtype: int64
0    0
Name: feature_cd__cat, dtype: int64
0    0
Name: feature_elektryczne-szyby-przednie__cat, dtyp

In [33]:
df.shape

(106290, 309)

In [36]:
cat_feats=[x for x in df.columns if SUFFIX in x]
cat_feats=[x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
X_cat= df[cat_feats].values
y_cat=df['price_value'].values
model2=DecisionTreeRegressor(max_depth=5)
scores=cross_val_score(model2,X_cat,y_cat,cv=3,scoring='neg_mean_absolute_error')

In [39]:
np.mean(scores)

-19566.588937368328

In [42]:
model3 = DecisionTreeRegressor(max_depth=5)
model3.fit(X_cat,y_cat)

imp=PermutationImportance(model3,random_state=0).fit(X_cat,y_cat)
eli5.show_weights(imp,feature_names=cat_feats)

Weight,Feature
0.2533 ± 0.0040,param_napęd__cat
0.2008 ± 0.0047,param_faktura-vat__cat
0.1943 ± 0.0088,param_stan__cat
0.1423 ± 0.0086,param_rok-produkcji__cat
0.0629 ± 0.0047,param_moc__cat
0.0424 ± 0.0014,feature_kamera-cofania__cat
0.0412 ± 0.0008,param_skrzynia-biegów__cat
0.0286 ± 0.0037,param_marka-pojazdu__cat
0.0191 ± 0.0022,param_pojemność-skokowa__cat
0.0163 ± 0.0005,feature_bluetooth__cat
